In [2]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
url =  "https://smithbrothersfurniture.com/style-details/?sid=211&piece=1"

In [65]:
req = requests.get(url)

In [66]:
req.status_code

200

In [67]:

soup = BeautifulSoup(req.content, "html.parser")

## Web pages html Code saving for selectors checking

In [43]:
# Saving the prettified HTML to a file
with open("product.html", "w", encoding="utf-8") as file:
    file.write(soup.prettify())

## Nav Bar links scraping

In [45]:
menu = soup.find("ul", class_="sub-menu")

In [46]:
categories = menu.find_all("a")

In [47]:
categories = [[item.text.strip(), item.get("href")] for item in categories]

In [48]:
def get_products(link):
    page = requests.get(link)
    products = []
    print("Getting products from : ", link)
    if page.status_code == 200:
        data = BeautifulSoup(page.content, "html.parser")
        products = data.find_all("div", class_ = "sb-col l3 m3 sb-mobile sb-center sb-hover-opacity sb-border-bottom")
        products = [item.find('a').get("href") for item in products if item.find('a')]
    
    if len(products) == 0:
        products = data.find_all('div', class_ = "ut-animated-image-item ut-image-gallery-item ut-animation-done")
        products = [item.find('a').get("href") for item in products if item.find('a')]
        

    if len(products) == 0:
            products = data.find_all('div', class_ = "sb-col l3 m3 sb-mobile sb-center sb-hover-opacity")
            products = [item.find('a').get("href") for item in products if item.find('a')]
        
    print("Products found : ", len(products))
    return products

In [ ]:
def get_products_links():
    url =  "https://smithbrothersfurniture.com/occasional-chairs/"
    req = requests.get(url)
    data = []
    if req.status_code == 200:
        soup = BeautifulSoup(req.content, "html.parser")
        menu = soup.find("ul", class_="sub-menu")
        categories = menu.find_all("a")
        categories = [[item.text.strip(), item.get("href")] for item in categories]
        
        for item in categories:
            category1 = item[0]
            if category1 != "BUILD YOUR OWN":
                products = get_products(item[1])
                for prod_link in products:
                    row = [category1, prod_link]
                    data.append(row)

    return data

In [ ]:
import csv
# Save data to CSV
csv_filename = "products.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Category", "Product URL"])  # Header
    writer.writerows(data)

print(f"Data saved to {csv_filename}")

## Product page scraping

In [50]:
sku = soup.find("span", class_ = "sb-style-no")
if sku:
    sku = sku.text.strip()
else:
    sku = ""

In [ ]:
sku

In [11]:
if sku:
    collection = sku + " " + "Style"
else:
    collection = ""

In [52]:
description = soup.find("p", class_ ="sb-style-desc")
if description:
    description = description.text.strip()
else:
    description = ""

In [53]:
description

''

In [54]:
catalog_pdf = soup.find("div", class_ = "sb-center sb-large")
if catalog_pdf:
    catalog_pdf = catalog_pdf.find("a").get("href")
else:
    catalog_pdf = ""

In [55]:
catalog_pdf

''

In [56]:
main_img = soup.find('img', class_ = "sb-style-details-image")
if main_img:
    main_img = main_img.get("src")
else:
    main_img = ""

In [57]:
main_img

''

In [68]:
product_imgs = soup.find_all("img", class_ = "sb-image sb-hover-opacity sb-wide-25 sb-margin-right")
if product_imgs:
    product_imgs = [item.get("src") for item in product_imgs]
else:
    product_imgs = []

In [70]:
product_imgs = soup.find_all("img", class_ = "sb-image sb-hover-opacity")
if product_imgs:
    product_imgs = [item.get("src") for item in product_imgs]
else:
    product_imgs = []

In [60]:
specifications = {}
specification_div = soup.find_all("div", class_ = "sb-row sb-mobile sb-margin-left")
if specification_div:
    for item in specification_div:
        divs = item.find_all("div")
        if len(divs) == 2:
            item_key = divs[0].find("strong")
            if item_key:
                item_key = item_key.text.strip().replace(":", "")
            else:
                continue
            item_value = divs[1].find("span")
            if item_value:
                item_value = item_value.text.strip().replace('"', "")
            else:
                continue

            if item_key and item_value:
                specifications[item_key] = item_value

if specifications:
    width = specifications.get("Width", None)
    depth = specifications.get("Depth", None)
    height = specifications.get("Height", None)
    specifications_str = "; ".join([f"{k}: {v}" for k, v in specifications.items()])
else:
    width = ""
    depth = ""
    height = ""
    specifications_str = ""
  


In [61]:
specifications

{}

In [62]:
specifications_str

''

In [ ]:
import pandas as pd

# Load the CSV file
file_path = "output/smith_brothers.csv"  # Update with your actual file path
df = pd.read_csv(file_path, dtype=str)

# Remove duplicate rows based on SKU (keeping the first occurrence)
df = df.drop_duplicates(subset=['SKU'], keep='first')

# Save the cleaned data back to a CSV file
df.to_csv("output/smith_brothers_updated.csv", index=False)

print("Duplicate rows removed based on SKU. Updated file saved as 'updated_file.csv'.")


In [28]:
import fitz

# Open the PDF file
pdf_path = "211.pdf"  # Change this to the correct path
doc = fitz.open(pdf_path)

for page in doc:
    text = page.get_text("text")  # Extract text from page
    lines = text.split("\n")  # Split text into lines

    extracted_text = []
    table_data = []
    is_table = False

    for line in lines:
        line = line.strip()
        
        # Detect the start of the table
        if "Width Depth Height Inside" in line or line.startswith("211-10"):
            is_table = True
        
        if is_table:
            if line:  # Avoid empty lines
                table_data.append(line)
            # Stop when a blank line appears after collecting table data
            elif table_data:
                break  
        else:
            extracted_text.append(line)

    break  # Stop after extracting the first page

# Print results
print("---- Text Before First Table ----")
print("\n".join(extracted_text))

print("\n---- First Table Data ----")
print("\n".join(table_data))



---- Text Before First Table ----
All dimensions are approximate.
211
FABRIC
s t yle
For more detailed information go to: www.smithbrothersfurniture.com
FEATURES:
PILLOWS:
One pair style #13, 18” (see price list for additional pillows)
BACK STYLE:
Semi-attached inside back
LEGS :
Built-in tapered legs (non-removable)
WOOD FINISH:
Shagbark (see samples for options)
NAILHEAD:
Small size (size/configuration shown standard)
Antique Brass shown (see samples for color selection)

DIMENSIONS:
Width
Depth
Height
Inside
Width
Seat
Height
Seat
Depth
Arm
Height

---- First Table Data ----
211-10
Sofa
80
39
37
66
20
21
26
211-20
Loveseat (not shown)
58
39
37
44
20
21
26
211-30
Chair
37
39
37
23
20
21
26
211-40
Ottoman
30
24
18
-
-
-
-


In [14]:
import fitz

# Open the PDF file
pdf_path = "211.pdf"  # Change this to the correct path
doc = fitz.open(pdf_path)

for page in doc:
    text = page.get_text("text")  # Extract text from page
    lines = text.split("\n")  # Split text into lines

    table_data = []
    is_table = False

    for line in lines:
        line = line.strip()
        
        # Detect the start of the table
        if "Width Depth Height Inside" in line or line.startswith("211-10"):
            is_table = True
        
        if is_table:
            if line:  # Avoid empty lines
                table_data.append(line)
            # Stop when a blank line appears after collecting table data
            elif table_data:
                break  

    break  # Stop after extracting the first page

# Clean and format the table data
formatted_table = "\n".join(table_data)

print("---- First Table Data ----")
print(formatted_table)


---- First Table Data ----
211-10
Sofa
80
39
37
66
20
21
26
211-20
Loveseat (not shown)
58
39
37
44
20
21
26
211-30
Chair
37
39
37
23
20
21
26
211-40
Ottoman
30
24
18
-
-
-
-


In [1]:
import fitz
import re

# Open the PDF file
pdf_path = "211.pdf"  # Change this to the correct path
doc = fitz.open(pdf_path)

for page in doc:
    text = page.get_text("text")  # Extract text from page
    lines = text.split("\n")  # Split text into lines

    table_data = []
    is_table = False

    for line in lines:
        line = line.strip()

        # Detect the start of the table using regex
        if re.match(r"^(SKU|\d{3,}-)", line):  # Matches "SKU" or "211-" or any similar pattern
            is_table = True

        if is_table:
            if line:  # Avoid empty lines
                table_data.append(line)
            # Stop when a blank line appears after collecting table data
            elif table_data:
                break  

    break  # Stop after extracting the first page

# Clean and format the table data
formatted_table = "\n".join(table_data)

print("---- First Table Data ----")
print(formatted_table)


---- First Table Data ----
211-10
Sofa
80
39
37
66
20
21
26
211-20
Loveseat (not shown)
58
39
37
44
20
21
26
211-30
Chair
37
39
37
23
20
21
26
211-40
Ottoman
30
24
18
-
-
-
-


In [5]:
import fitz

# Open the PDF file
pdf_path = "211.pdf"  # Change this to the correct path
doc = fitz.open(pdf_path)

for page in doc:
    text = page.get_text("text")  # Extract text from page
    lines = text.split("\n")  # Split text into lines

    table_data = []
    is_table = False

    for line in lines:
        line = line.strip()

        # Detect the start of the table using regex
        if re.match(r"^[A-Z]?\d{3,}-\d{2,}\*?$", line.strip()):
            is_table = True

        if is_table:
            if line:  # Avoid empty lines
                table_data.append(line)
            # Stop when a blank line appears after collecting table data
            elif table_data:
                break  

    break  # Stop after extracting the first page

# Clean and format the table data
formatted_table = "\n".join(table_data)

print("---- First Table Data ----")
print(formatted_table)


---- First Table Data ----
211-10
Sofa
80
39
37
66
20
21
26
211-20
Loveseat (not shown)
58
39
37
44
20
21
26
211-30
Chair
37
39
37
23
20
21
26
211-40
Ottoman
30
24
18
-
-
-
-


## The right code

In [9]:
import fitz

# Open the PDF file
pdf_path = "M100.pdf"  # Change this to the correct path
doc = fitz.open(pdf_path)

for page in doc:
    text = page.get_text("text")  # Extract text from page
    lines = text.split("\n")  # Split text into lines

    table_data = []
    is_table = False

    for line in lines:
        line = line.strip()

        # Detect the start of the table using regex
        if re.match(r"^[A-Z]?\d{3,}-\d{2,}\*?$", line.strip()):
            is_table = True

        if is_table:
            # Stop collecting lines when "*Battery Pack Available" is found
            if "*Battery Pack Available" in line:
                break  
            if line:  # Avoid empty lines
                table_data.append(line)
            # Stop when a blank line appears after collecting table data
            elif table_data:
                break  

    break  # Stop after extracting the first page

# Clean and format the table data
formatted_table = "\n".join(table_data)

print("---- First Table Data ----")
# print(formatted_table)
for l in table_data:
    print(l)
    # print("--------------------------------")


---- First Table Data ----
M100-84*
LAF Motorized Reclining Chair with
Headrest (shown)
34
41
44
26
20
21
25
70
5
18
M100-85*
RAF Motorized Reclining Chair with
Headrest (shown)
34
41
44
26
20
21
25
70
5
18
M100-36
Stationary Armless Chair (shown)
27
41
44
27
20
21
-
-
-
-
M100-86*
Motorized Reclining Armless Chair with
Headrest (shown)
27
41
44
27
20
21
-
70
5
18
M100-53
Wedge (shown)
81
41
44
-
20
21
-
-
-
-
M100-71
Console
18
40
24
-
-
-
-
-
-
-
WC-101
Wood Console Tray
19
21
4
-
-
-
-
-
-
-
M100-83*
Motorized Reclining Chair with
Headrest
40
41
44
24
20
21
25
70
5
18
M100-90*
Motorized Glider Reclining Chair with
Headrest (shown)
40
41
44
24
20
21
25
66
12
17


## Updated version

In [12]:
import fitz

# Open the PDF file
pdf_path = "M100.pdf"  # Change this to the correct path
doc = fitz.open(pdf_path)

for page in doc:
    text = page.get_text("text")  # Extract text from page
    lines = text.split("\n")  # Split text into lines

    table_data = []
    extracted_text = []
    features = []
    is_table = False

    for line in lines:
        line = line.strip()

        # Detect the start of the table using regex
        if re.match(r"^[A-Z]?\d{3,}-\d{2,}\*?$", line.strip()):
            is_table = True

        if is_table:
            # Stop collecting lines when "*Battery Pack Available" is found
            if "*Battery Pack Available" in line:
                break  
            if line:  # Avoid empty lines
                table_data.append(line)
            # Stop when a blank line appears after collecting table data
            elif table_data:
                break 
        else:
            if "DIMENSIONS:" in line:
                features = extracted_text
            extracted_text.append(line) 
            if "FEATURES:" in line:
                extracted_text = []
  

    break  # Stop after extracting the first page

# Clean and format the table data
formatted_table = "\n".join(table_data)



print("---- Text Before First Table ----")
features_filtered = []
for item in features:
    if "DIMENSIONS:" in item:
        break
    features_filtered.append(item)
print("\n".join(features_filtered))

print("\n---- First Table Data ----")
print("\n".join(table_data))


---- Text Before First Table ----
BACK STYLE:
Bustle back with motorized adjustable headrest
NAILHEAD:
Small size (size/configuration shown standard)
Brushed Nickel shown (see samples for color selection)
WOOD FINISH:
The Console features an optional wood tray (see price list for limited wood finish color options)
DESIGN NOTE:
Due to the complex seat design, collaging is not permitted.
Activator switches on recliners feature a charging port.
The Motorized Reclining Armless Chair with Headrest (-86) features a corded hand control.
The Console features a pop-up charging station, cup holders, and drawer.

---- First Table Data ----
M100-84*
LAF Motorized Reclining Chair with
Headrest (shown)
34
41
44
26
20
21
25
70
5
18
M100-85*
RAF Motorized Reclining Chair with
Headrest (shown)
34
41
44
26
20
21
25
70
5
18
M100-36
Stationary Armless Chair (shown)
27
41
44
27
20
21
-
-
-
-
M100-86*
Motorized Reclining Armless Chair with
Headrest (shown)
27
41
44
27
20
21
-
70
5
18
M100-53
Wedge (shown)
8

In [5]:
import fitz
import requests
import io
import re

# URL of the PDF file
pdf_url = "https://smithbrothersfurniture.com/downloads/CatalogPages/M101.pdf"

# Download the PDF
response = requests.get(pdf_url)
response.raise_for_status()  # Ensure we got the file successfully

# Open the PDF from memory
doc = fitz.open(stream=io.BytesIO(response.content), filetype="pdf")

for page in doc:
    text = page.get_text("text")  # Extract text from page
    lines = text.split("\n")  # Split text into lines

    table_data = []
    is_table = False

    for line in lines:
        line = line.strip()

        # Detect the start of the table using regex
        if re.match(r"^[A-Z]?\d{3,}-\d{2,}\*?$", line):
            is_table = True

        if is_table:
            # Stop collecting lines when "*Battery Pack Available" is found
            if "*Battery Pack Available" in line:
                break  
            if line:  # Avoid empty lines
                table_data.append(line)
            # Stop when a blank line appears after collecting table data
            elif table_data:
                break  

    break  # Stop after extracting the first page

# Clean and format the table data
formatted_table = "\n".join(table_data)

print("---- First Table Data ----")
for l in table_data:
    print(l)


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


---- First Table Data ----
M101-84*
LAF Motorized Reclining Chair with
Headrest (shown)
32
41
44
26
20
21
25
70
5
18
M101-85*
RAF Motorized Reclining Chair with
Headrest (shown)
32
41
44
26
20
21
25
70
5
18
M101-36
Stationary Armless Chair
27
41
44
27
20
21
-
-
-
-
M101-86*
Motorized Reclining Armless Chair
with Headrest
27
41
44
27
20
21
-
70
5
18
M101-53
Wedge
81
41
44
-
20
21
-
-
-
-
M101-71
Console (shown)
18
40
24
-
-
-
-
-
-
-
WC-101
Wood Console Tray
19
21
4
-
-
-
-
-
-
-
M101-83*
Motorized Reclining Chair with
Headrest
36
41
44
24
20
21
25
70
5
18
M101-90*
Motorized Glider Reclining Chair
with Headrest (shown)
36
41
44
24
20
21
25
66
12
17


# PDF Data processing

In [ ]:
import pandas as pd
import fitz
import requests
import io
import re

class CatalogProcessor:
    def __init__(self, csv_file, output_file):
        self.csv_file = csv_file
        self.output_file = output_file
        self.data = None

    def load_csv(self):
        """Load the CSV file into a Pandas DataFrame."""
        self.data = pd.read_csv(self.csv_file, dtype = str).fillna("")
    
    def extract_pdf_data(self, pdf_url):
        """Extract table data from the given PDF URL."""
        try:
            response = requests.get(pdf_url)
            response.raise_for_status()
            doc = fitz.open(stream=io.BytesIO(response.content), filetype="pdf")

            for page in doc:
                text = page.get_text("text")
                lines = text.split("\n")

                table_data = []
                is_table = False
                features = []
                extracted_text = []

                for line in lines:
                    line = line.strip()

                    if re.match(r"^[A-Z]?\d{3,}-\d{2,}\*?$", line):
                        is_table = True

                    if is_table:
                        if "*Battery Pack Available" in line:
                            break
                        if line:
                            table_data.append(line)
                        elif table_data:
                            break  
                    else:
                        if "DIMENSIONS:" in line:
                            features = extracted_text
                        extracted_text.append(line) 
                        if "FEATURES:" in line:
                            extracted_text = []

                features_filtered = []
                for item in features:
                    if "DIMENSIONS:" in item:
                        break
                    features_filtered.append(item)
                features_data = 
                

                return {
                    "features_data" : "\n".join(features_filtered),
                    "dimensions_table": table_data
                }
            
        except requests.RequestException as e:
            print(f"Failed to download PDF: {e}")
            return ""
    
    def extract_photos(self, row):
        """Extract all non-empty PHOTO columns from a row."""
        return [row[f"PHOTO{i}"] for i in range(1, 11) if row.get(f"PHOTO{i}", "").strip()]
    
    def process_data(self):
        """Process each row to extract PDF data and photos, then save the result."""
        new_rows = []
        for _, row in self.data.iterrows():
            if row["CATALOG_PDF"]:
                pdf_data =  self.extract_pdf_data(row["CATALOG_PDF"])
                construction_text =  pdf_data["features_data"]
                photos = self.extract_photos(row)

                for idx, photo in enumerate(photos, start=1):
                    new_row = row.copy()
                    new_row["SKU"] = f"{row['SKU']}-{idx}"
                    new_row["PHOTO1"] = photo
                    new_row["CONSTRUCTION"] = construction_text
                    new_rows.append(new_row)
                    print(new_row)

        new_df = pd.DataFrame(new_rows)
        new_df.to_csv(self.output_file, index=False)

if __name__ == "__main__":
    processor = CatalogProcessor("output/smith_brothers.csv", "output/smith_brothers_sku_updated.csv")
    processor.load_csv()
    processor.process_data()


KeyboardInterrupt: 